---
title: "Stock Explorer"
author: "Ty Coon"
format: 
  dashboard:
    theme: [lux]

# author: garretgman accessed 17th April 2025
# https://github.com/jjallaire/stock-explorer-dashboard/blob/main/dashboard.qmd
---

In [ ]:
#| echo: false
import time
import requests_cache
import yfinance as yf

# 1) Cache every HTTP request for 15 minutes
requests_cache.install_cache("yf_cache", expire_after=900)

# 2) Monkey‑patch Ticker.info so it always sleeps 1 s after fetching
_orig_info = yf.Ticker.info.fget
def _throttled_info(self):
    data = _orig_info(self)
    time.sleep(1)
    return data
yf.Ticker.info = property(_throttled_info)

In [ ]:
# to render w/ a different ticker parameter:
# quarto render dashboard.qmd -P ticker:GOOG

ticker = "KKR"

In [ ]:
import helpers
stock = helpers.get_stock(ticker)
data = helpers.get_data(stock)

## Row


In [ ]:
from IPython.display import Markdown
Markdown("**{symbol}** --- {longName} --- [{website}]({website})"
    .format(**stock.info))

## Row


In [ ]:
dict(
    icon = "currency-dollar",
    color = "secondary",
    value = helpers.get_price(data)
)

In [ ]:
change = helpers.get_change(data)
dict(
    value = change['amount'],
    icon = change['icon'],
    color = change['color']
) 

In [ ]:
dict(
    icon = "percent",
    color = "light",
    value = change['percent'],
) 

## Row

### Column {width=75%}


In [ ]:
helpers.make_candlestick_chart(data, ticker.upper())

### Column {width=25%}


In [ ]:
ohlc = helpers.make_OHLC_table(data)

::: {.card}

#### Last Close: `{python} ohlc['date']`

| Close	 | `{python} ohlc['close']`  |
|:-------|--------------------------:|
| Open	 | `{python} ohlc['open']`   |
| High	 | `{python} ohlc['high']`   |
| Low	 | `{python} ohlc['low']`    |
| Volume | `{python} ohlc['volume']` |
: {.striped}

:::